# Supply Chain Demand Forecasting using Prophet

**Copyright (c) 2024 Shrikara Kaudambady. All rights reserved.**

This notebook demonstrates how to build a time series forecasting model for supply chain demand prediction using the Prophet library developed by Facebook.

### 1. Installing and Importing Libraries

In [ ]:
!pip install pandas prophet matplotlib scikit-learn

In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import io
import matplotlib.pyplot as plt

print("Libraries imported successfully.")

### 2. Data Generation and Loading

For this demonstration, we will generate a synthetic dataset that mimics daily sales data for a product over a few years. This makes the notebook self-contained.

The data will have a clear trend, yearly and weekly seasonality, and some noise.

In [ ]:
# Generate synthetic sales data
date_rng = pd.date_range(start='2020-01-01', end='2023-12-31', freq='D')
np.random.seed(42)

# Trend: linear growth
trend = np.arange(len(date_rng)) * 0.5

# Seasonality: yearly and weekly
yearly_seasonality = 100 * (1 + np.sin(np.arange(len(date_rng)) * 2 * np.pi / 365.25))
weekly_seasonality = 20 * (1 + np.cos(date_rng.dayofweek * 2 * np.pi / 7))

# Noise
noise = np.random.normal(0, 20, len(date_rng))

# Combine components
sales = 500 + trend + yearly_seasonality + weekly_seasonality + noise

# Create DataFrame
sales_data = pd.DataFrame({'Date': date_rng, 'Sales': sales})
sales_data['Sales'] = sales_data['Sales'].astype(int)

print("Synthetic dataset created. Here are the first 5 rows:")
sales_data.head()

### 3. Exploratory Data Analysis (EDA)

Let's visualize the data to understand its components.

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(sales_data['Date'], sales_data['Sales'])
plt.title('Daily Product Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.grid(True)
plt.show()

### 4. Preparing Data for Prophet

Prophet requires the data to be in a specific format with two columns:
- `ds`: The timestamp column (must be datetime).
- `y`: The value we want to forecast (in this case, 'Sales').

In [ ]:
df_prophet = sales_data.rename(columns={'Date': 'ds', 'Sales': 'y'})
df_prophet.head()

### 5. Training the Prophet Model

We'll split the data into a training set (first 3 years) and a test set (the last year) to evaluate our model's performance.

In [ ]:
train_data = df_prophet[df_prophet['ds'] < '2023-01-01']
test_data = df_prophet[df_prophet['ds'] >= '2023-01-01']

print(f"Training data size: {len(train_data)}")
print(f"Test data size: {len(test_data)}")

# Initialize and fit the model
# Prophet automatically detects yearly and weekly seasonality.
model = Prophet(daily_seasonality=False) # Daily is off as it can be noisy; weekly/yearly are on by default
model.fit(train_data)

### 6. Making Future Predictions

We create a 'future' DataFrame that contains the dates for which we want a forecast. Here, we'll predict for the entire year of 2023.

In [ ]:
future = model.make_future_dataframe(periods=365) # Forecast for the next 365 days
forecast = model.predict(future)

print("Forecast generated. Here are the last few rows:")
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

### 7. Visualizing the Forecast

Prophet has built-in plotting functions to visualize the forecast.

In [ ]:
fig1 = model.plot(forecast)
plt.title('Demand Forecast for 2023')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.show()

In [ ]:
# Plot the forecast components (trend, weekly, yearly seasonality)
fig2 = model.plot_components(forecast)
plt.show()

### 8. Evaluating Model Performance

Let's compare the predicted values (`yhat`) with the actual values (`y`) from our test set (the year 2023).

In [ ]:
# Merge forecast with actuals
performance = pd.merge(test_data, forecast[['ds', 'yhat']], on='ds')

# Calculate metrics
mae = mean_absolute_error(performance['y'], performance['yhat'])
mse = mean_squared_error(performance['y'], performance['yhat'])
rmse = np.sqrt(mse)

print(f"Model Performance on Test Set (2023):")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

# Plotting actual vs predicted
plt.figure(figsize=(16, 6))
plt.plot(performance['ds'], performance['y'], label='Actual Sales')
plt.plot(performance['ds'], performance['yhat'], label='Predicted Sales', linestyle='--')
plt.title('Actual vs. Predicted Sales for 2023')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.show()

### 9. Conclusion

This notebook demonstrates a complete workflow for demand forecasting using Prophet.

The model successfully captured the upward trend and the seasonal patterns in the data, providing a reliable forecast. The evaluation metrics (like MAE and RMSE) give us a quantitative measure of the forecast's accuracy.

This forecast can be used by supply chain managers to:
- Optimize inventory levels to reduce holding costs and avoid stockouts.
- Plan procurement and logistics activities.
- Make informed decisions about marketing campaigns and promotions.

**Copyright (c) 2024 Shrikara Kaudambady. All rights reserved.**